In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [34]:
data=pd.read_csv('../data/clean_data.csv')
data.head()
# Observamos que las variables categóricas son de las columnas 'Offer Accepted', 'Reward', 'Mailer Type', 'Income Level', 'Overdraft Protection', 'Credit Rating' y 'Own Your Home'.
# Estas variables categóricas las tenemos que pasar a numéricas.

,Unnamed: 0,Customer Number,Offer Accepted,Reward,Mailer Type,Income Level,# Bank Accounts Open,Overdraft Protection,Credit Rating,# Credit Cards Held,# Homes Owned,Household Size,Own Your Home,Average Balance,Q1 Balance,Q2 Balance,Q3 Balance,Q4 Balance
0,0,1,No,Air Miles,Letter,High,1,No,High,2,1,4,No,1160.75,1669.0,877.0,1095.0,1002.0
1,1,2,No,Air Miles,Letter,Medium,1,No,Medium,2,2,5,Yes,147.25,39.0,106.0,78.0,366.0
2,2,3,No,Air Miles,Postcard,High,2,No,Medium,2,1,2,Yes,276.50,367.0,352.0,145.0,242.0
3,3,4,No,Air Miles,Letter,Medium,2,No,High,1,1,4,No,1219.00,1578.0,1760.0,1119.0,419.0
4,4,5,No,Air Miles,Letter,Medium,1,No,Medium,2,1,6,Yes,1211.00,2140.0,1357.0,982.0,365.0


In [35]:
data['Offer Accepted'].unique()

array(['No', 'Yes'], dtype=object)

In [24]:
#Columna Offer Accepted
le = LabelEncoder().fit(data['Offer Accepted'])
data['Offer Accepted'] = le.transform(data['Offer Accepted'])
data['Offer Accepted'].unique()

array([0, 1])

In [36]:
data['Reward'].unique()

array(['Air Miles', 'Cash Back', 'Points'], dtype=object)

In [25]:
#Columna Reward
le = LabelEncoder().fit(data['Reward'])
data['Reward'] = le.transform(data['Reward'])
data['Reward'].unique()

array([0, 1, 2])

In [37]:
data['Mailer Type'].unique()

array(['Letter', 'Postcard'], dtype=object)

In [26]:
#Columna Mailer Type
le = LabelEncoder().fit(data['Mailer Type'])
data['Mailer Type'] = le.transform(data['Mailer Type'])
data['Mailer Type'].unique()

array([0, 1])

In [38]:
data['Income Level'].unique()

array(['High', 'Medium', 'Low'], dtype=object)

In [29]:
#Columna Mailer Type
le = LabelEncoder().fit(data['Income Level'])
data['Income Level'] = le.transform(data['Income Level'])
data['Income Level'].unique()

array([0, 2, 1])

In [39]:
data['Overdraft Protection'].unique()

array(['No', 'Yes'], dtype=object)

In [30]:
#Columna Overdraft Protection
le = LabelEncoder().fit(data['Overdraft Protection'])
data['Overdraft Protection'] = le.transform(data['Overdraft Protection'])
data['Overdraft Protection'].unique()

array([0, 1])

In [40]:
data['Credit Rating'].unique()

array(['High', 'Medium', 'Low'], dtype=object)

In [31]:
#Columna Credit Rating
le = LabelEncoder().fit(data['Credit Rating'])
data['Credit Rating'] = le.transform(data['Credit Rating'])
data['Credit Rating'].unique()

array([0, 2, 1])

In [41]:
data['Own Your Home'].unique()

array(['No', 'Yes'], dtype=object)

In [32]:
#Columna Own Your Home
le = LabelEncoder().fit(data['Own Your Home'])
data['Own Your Home'] = le.transform(data['Own Your Home'])
data['Own Your Home'].unique()

array([0, 1])

In [ ]:

credit_card_dummy = pd.get_dummies(data, drop_first = True)

X = credit_card_dummy.drop('Offer Accepted', axis = True)
y = credit_card_dummy['Offer Accepted']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify = y)


y_train.value_counts(normalize = True), y_test.value_counts(normalize = True)


lg = LogisticRegression(max_iter = 2000)
lg.fit(X_train, y_train)
pred = lg.predict(X_test)


acc = accuracy_score(y_test, pred)
print(f'Accuracy lg: {acc}')

cm = confusion_matrix(y_test, pred)

plt.figure(figsize = (15, 20))
sns.heatmap(cm, annot = True)
plt.show()